In [ ]:
#پیش بینی آب و هوا مبنتی بر یادگیری ماشین

!pip install --upgrade --no-cache-dir gdown
!gdown 18O248Vc5mnWoB21w77NIlFYlbPDsE9tw

import pandas as pd
import numpy as np

df = pd.read_csv("weather_prediction_dataset.csv")

Downloading...
From: https://drive.google.com/uc?id=18O248Vc5mnWoB21w77NIlFYlbPDsE9tw
To: /content/weather_prediction_dataset.csv
100% 2.77M/2.77M [00:00<00:00, 69.1MB/s]


In [ ]:
france_cities = ["TOURS"]

columns_to_keep = ["DATE", "MONTH"] + [col for col in df.columns if any(city in col for city in france_cities)]
df_france = df[columns_to_keep]

df_france.to_csv("weather_prediction_dataset_france.csv", index=False)

print("Filtered dataset saved as 'weather_prediction_dataset_france.csv'")




Filtered dataset saved as 'weather_prediction_dataset_france.csv'


In [ ]:
from google.colab import files

# Download the filtered dataset
files.download("weather_prediction_dataset_france.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

file_path = "weather_prediction_dataset_france.csv"
df = pd.read_csv(file_path)

df.fillna(df.mean(), inplace=True)

relevant_features = ["DATE", "MONTH", "TOURS_temp_mean", "TOURS_humidity",
                     "TOURS_pressure", "TOURS_wind_speed"]
df_selected = df[relevant_features]

scaler = MinMaxScaler()
df_scaled = df_selected.copy()
df_scaled.iloc[:, 2:] = scaler.fit_transform(df_selected.iloc[:, 2:])

df_scaled.to_csv("weather_prediction_dataset_france_final.csv", index=False)

print("Preprocessing completed! Saved:")
print("- 'weather_prediction_dataset_france_final.csv' (scaled data)")

Preprocessing completed! Saved:
- 'weather_prediction_dataset_france_final.csv' (scaled data)


In [ ]:
# Download the preprocessed dataset
files.download("weather_prediction_dataset_france_final.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

file_path = "weather_prediction_dataset_france_final.csv"
df = pd.read_csv(file_path)

df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')

test_data = df[df['DATE'].dt.year == 2009]
train_data = df[df['DATE'].dt.year != 2009]

train_data = train_data.drop(columns=['DATE', 'MONTH'])
test_data = test_data.drop(columns=['DATE', 'MONTH'])

X_train, Y_train = train_data.iloc[:, :-1], train_data.iloc[:, -1]
X_test, Y_test = test_data.iloc[:, :-1], test_data.iloc[:, -1]

scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_Y = MinMaxScaler()
Y_train_scaled = scaler_Y.fit_transform(Y_train.values.reshape(-1, 1))
Y_test_scaled = scaler_Y.transform(Y_test.values.reshape(-1, 1))

def create_sliding_window(data_X, data_Y, window_size, overlap):
    step = window_size - overlap
    X, Y = [], []
    for i in range(0, len(data_X) - window_size + 1, step):
        X.append(data_X[i:i+window_size, :])
        Y.append(data_Y[i+window_size-1, :])
    return np.array(X), np.array(Y)

window_size = 5
overlap = 4

Y_test_scaled = np.clip(Y_test_scaled, 0, 1)

X_train_windowed, Y_train_windowed = create_sliding_window(X_train_scaled, Y_train_scaled, window_size, overlap)
X_test_windowed, Y_test_windowed = create_sliding_window(X_test_scaled, Y_test_scaled, window_size, overlap)

np.save("X_train.npy", X_train_windowed)
np.save("Y_train.npy", Y_train_windowed)
np.save("X_test.npy", X_test_windowed)
np.save("Y_test.npy", Y_test_windowed)

print("🚀 Sliding window train/test data created successfully!")


🚀 Sliding window train/test data created successfully!


In [ ]:
print("X_train shape:", X_train_windowed.shape)
print("Y_train shape:", Y_train_windowed.shape)
print("X_test shape:", X_test_windowed.shape)
print("Y_test shape:", Y_test_windowed.shape)

X_train shape: (3285, 5, 3)
Y_train shape: (3285, 1)
X_test shape: (361, 5, 3)
Y_test shape: (361, 1)


In [ ]:
#MLR

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

X_train = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")
X_test = np.load("X_test.npy")
Y_test = np.load("Y_test.npy")

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

mlr = LinearRegression()
mlr.fit(X_train, Y_train)

Y_pred_train = np.clip(mlr.predict(X_train), 0, 1)
Y_pred_test = np.clip(mlr.predict(X_test), 0, 1)

Y_train_pred_inv = scaler_Y.inverse_transform(Y_pred_train)
Y_test_pred_inv = scaler_Y.inverse_transform(Y_pred_test)
Y_train_actual_inv = scaler_Y.inverse_transform(Y_train)
Y_test_actual_inv = scaler_Y.inverse_transform(Y_test)

train_rmse = np.sqrt(mean_squared_error(Y_train_actual_inv, Y_train_pred_inv))
test_rmse = np.sqrt(mean_squared_error(Y_test_actual_inv, Y_test_pred_inv))
train_mae = mean_absolute_error(Y_train_actual_inv, Y_train_pred_inv)
test_mae = mean_absolute_error(Y_test_actual_inv, Y_test_pred_inv)
train_mse = mean_squared_error(Y_train_actual_inv, Y_train_pred_inv)
test_mse = mean_squared_error(Y_test_actual_inv, Y_test_pred_inv)

print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
print(f"Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
print(f"Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}")

Train RMSE: 0.1459, Test RMSE: 0.1402
Train MAE: 0.1162, Test MAE: 0.1150
Train MSE: 0.0213, Test MSE: 0.0197


In [ ]:
#MPR

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

X_train = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")
X_test = np.load("X_test.npy")
Y_test = np.load("Y_test.npy")

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

def train_polynomial_regression(X_train, Y_train, X_test, scaler_Y, degree=2, alpha=5):
    poly = PolynomialFeatures(degree=degree, interaction_only=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    scaler_X = MinMaxScaler()
    X_train_poly = scaler_X.fit_transform(X_train_poly)
    X_test_poly = scaler_X.transform(X_test_poly)

    model = Ridge(alpha=alpha)
    model.fit(X_train_poly, Y_train)

    Y_train_pred = scaler_Y.inverse_transform(model.predict(X_train_poly).reshape(-1, 1))
    Y_test_pred = scaler_Y.inverse_transform(model.predict(X_test_poly).reshape(-1, 1))

    return model, Y_train_pred, Y_test_pred


mpr_model, Y_train_pred_inv, Y_test_pred_inv = train_polynomial_regression(
    X_train, Y_train, X_test, scaler_Y, degree=2, alpha=5)

Y_train_actual_inv = scaler_Y.inverse_transform(Y_train_windowed.reshape(-1, 1))
Y_test_actual_inv = scaler_Y.inverse_transform(Y_test_windowed.reshape(-1, 1))


train_rmse = np.sqrt(mean_squared_error(Y_train_actual_inv, Y_train_pred_inv))
test_rmse = np.sqrt(mean_squared_error(Y_test_actual_inv, Y_test_pred_inv))
train_mae = mean_absolute_error(Y_train_actual_inv, Y_train_pred_inv)
test_mae = mean_absolute_error(Y_test_actual_inv, Y_test_pred_inv)
train_mse = mean_squared_error(Y_train_actual_inv, Y_train_pred_inv)
test_mse = mean_squared_error(Y_test_actual_inv, Y_test_pred_inv)

print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
print(f"Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
print(f"Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}")



Train RMSE: 0.1426, Test RMSE: 0.1357
Train MAE: 0.1142, Test MAE: 0.1105
Train MSE: 0.0203, Test MSE: 0.0184


In [ ]:
#KNN
from sklearn.neighbors import KNeighborsRegressor

X_train = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")
X_test = np.load("X_test.npy")
Y_test = np.load("Y_test.npy")

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

scaler_Y = MinMaxScaler()
Y_train = scaler_Y.fit_transform(Y_train)
Y_test = scaler_Y.transform(Y_test)

def train_knn(X_train, Y_train, X_test, scaler_Y, n_neighbors=5):
    knn = KNeighborsRegressor(n_neighbors=n_neighbors)
    knn.fit(X_train, Y_train.ravel())

    Y_train_pred = scaler_Y.inverse_transform(knn.predict(X_train).reshape(-1, 1))
    Y_test_pred = scaler_Y.inverse_transform(knn.predict(X_test).reshape(-1, 1))

    return knn, Y_train_pred, Y_test_pred

knn_model, Y_train_pred_inv, Y_test_pred_inv = train_knn(X_train, Y_train, X_test, scaler_Y, n_neighbors=5)

Y_train_actual_inv = scaler_Y.inverse_transform(Y_train)
Y_test_actual_inv = scaler_Y.inverse_transform(Y_test)

train_rmse = np.sqrt(mean_squared_error(Y_train_actual_inv, Y_train_pred_inv))
test_rmse = np.sqrt(mean_squared_error(Y_test_actual_inv, Y_test_pred_inv))
train_mae = mean_absolute_error(Y_train_actual_inv, Y_train_pred_inv)
test_mae = mean_absolute_error(Y_test_actual_inv, Y_test_pred_inv)
train_mse = mean_squared_error(Y_train_actual_inv, Y_train_pred_inv)
test_mse = mean_squared_error(Y_test_actual_inv, Y_test_pred_inv)

print(f"KNN Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
print(f"KNN Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
print(f"KNN Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}")



KNN Train RMSE: 0.1256, Test RMSE: 0.1452
KNN Train MAE: 0.0993, Test MAE: 0.1175
KNN Train MSE: 0.0158, Test MSE: 0.0211


In [ ]:
# MLP

from tensorflow import keras

X_train = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")
X_test = np.load("X_test.npy")
Y_test = np.load("Y_test.npy")

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

def train_mlp(X_train, Y_train, X_test, scaler_Y, epochs=100, batch_size=32):
    input_dim = X_train.shape[1]

    model = keras.Sequential([
        keras.layers.Input(shape=(input_dim,)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1, activation='linear')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])

    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.1)

    Y_train_pred = scaler_Y.inverse_transform(model.predict(X_train).reshape(-1, 1))
    Y_test_pred = scaler_Y.inverse_transform(model.predict(X_test).reshape(-1, 1))

    return model, Y_train_pred, Y_test_pred

mlp_model, Y_train_pred_inv, Y_test_pred_inv = train_mlp(
    X_train, Y_train, X_test, scaler_Y, epochs=100, batch_size=32)

Y_train_actual_inv = scaler_Y.inverse_transform(Y_train.reshape(-1, 1))
Y_test_actual_inv = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))

train_rmse = np.sqrt(mean_squared_error(Y_train_actual_inv, Y_train_pred_inv))
test_rmse = np.sqrt(mean_squared_error(Y_test_actual_inv, Y_test_pred_inv))
train_mae = mean_absolute_error(Y_train_actual_inv, Y_train_pred_inv)
test_mae = mean_absolute_error(Y_test_actual_inv, Y_test_pred_inv)
train_mse = mean_squared_error(Y_train_actual_inv, Y_train_pred_inv)
test_mse = mean_squared_error(Y_test_actual_inv, Y_test_pred_inv)

print(f"MLP Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
print(f"MLP Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
print(f"MLP Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}")



Epoch 1/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0259 - mae: 0.1287 - val_loss: 0.0270 - val_mae: 0.1348
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0226 - mae: 0.1203 - val_loss: 0.0214 - val_mae: 0.1125
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0233 - mae: 0.1218 - val_loss: 0.0213 - val_mae: 0.1123
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0228 - mae: 0.1210 - val_loss: 0.0233 - val_mae: 0.1218
Epoch 5/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0226 - mae: 0.1195 - val_loss: 0.0225 - val_mae: 0.1184
Epoch 6/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0211 - mae: 0.1164 - val_loss: 0.0243 - val_mae: 0.1252
Epoch 7/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0228 - mae: 0.1201 - val_loss: 0.0228 - val_mae: 0.1201
Epoch 8/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0237 - mae: 0.1230 - val_loss: 0.0210 - val_mae: 0.1113
Epoch 9/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.021

In [ ]:
#CNN

def create_sliding_window(X, Y, window_size=5, step=1):
    X_windowed, Y_windowed = [], []
    for i in range(0, len(X) - window_size + 1, step):
        X_windowed.append(X[i:i+window_size])
        Y_windowed.append(Y[i+window_size-1])
    return np.array(X_windowed), np.array(Y_windowed)

def train_cnn(X_train, Y_train, X_test, scaler_Y, epochs=100, batch_size=32):
    time_steps, num_features = X_train.shape[1], X_train.shape[2]

    model = keras.Sequential([
        keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, num_features)),
        keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='linear')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])

    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.1)

    Y_train_pred = scaler_Y.inverse_transform(model.predict(X_train))
    Y_test_pred = scaler_Y.inverse_transform(model.predict(X_test))

    return model, Y_train_pred, Y_test_pred


window_size, step = 5, 1
X_train_windowed, Y_train_windowed = create_sliding_window(X_train, Y_train, window_size, step)
X_test_windowed, Y_test_windowed = create_sliding_window(X_test, Y_test, window_size, step)

X_train_windowed = X_train_windowed.reshape((X_train_windowed.shape[0], window_size, X_train.shape[1]))
X_test_windowed = X_test_windowed.reshape((X_test_windowed.shape[0], window_size, X_test.shape[1]))

cnn_model, Y_train_pred_inv, Y_test_pred_inv = train_cnn(
    X_train_windowed, Y_train_windowed, X_test_windowed, scaler_Y, epochs=100, batch_size=32)

Y_train_actual_inv = scaler_Y.inverse_transform(Y_train_windowed.reshape(-1, 1))
Y_test_actual_inv = scaler_Y.inverse_transform(Y_test_windowed.reshape(-1, 1))

train_rmse = np.sqrt(mean_squared_error(Y_train_actual_inv, Y_train_pred_inv))
test_rmse = np.sqrt(mean_squared_error(Y_test_actual_inv, Y_test_pred_inv))
train_mae = mean_absolute_error(Y_train_actual_inv, Y_train_pred_inv)
test_mae = mean_absolute_error(Y_test_actual_inv, Y_test_pred_inv)
train_mse = mean_squared_error(Y_train_actual_inv, Y_train_pred_inv)
test_mse = mean_squared_error(Y_test_actual_inv, Y_test_pred_inv)

print(f"CNN Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
print(f"CNN Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
print(f"CNN Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0931 - mae: 0.2199 - val_loss: 0.0217 - val_mae: 0.1141
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0253 - mae: 0.1269 - val_loss: 0.0229 - val_mae: 0.1193
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0254 - mae: 0.1278 - val_loss: 0.0218 - val_mae: 0.1147
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0258 - mae: 0.1275 - val_loss: 0.0227 - val_mae: 0.1188
Epoch 5/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0239 - mae: 0.1229 - val_loss: 0.0232 - val_mae: 0.1207
Epoch 6/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0245 - mae: 0.1259 - val_loss: 0.0220 - val_mae: 0.1155
Epoch 7/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0244 - mae: 0.1241 - val_loss: 0.0223 - val_mae: 0.1169
Epoch 8/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0246 - mae: 0.1251 - val_loss: 0.0224 - val_mae: 0.1170
Epoch 9/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0237 - mae: 0.1

In [ ]:
from tqdm import tqdm

df = pd.read_csv("weather_prediction_dataset.csv")

print("Dataset Columns:", df.columns.tolist())

cities = set()
features = set()
for col in df.columns:
    if "_" in col:
        city, feature = col.split("_", 1)
        cities.add(city)
        features.add(feature)

cities = list(cities)
features = list(features)

feature_columns = [f"{city}_{feature}" for city in cities for feature in features if f"{city}_{feature}" in df.columns]

df = df[['DATE'] + feature_columns]

df['DATE'] = pd.to_datetime(df['DATE'])
df = df.sort_values(by='DATE')

df.fillna(df.mean(), inplace=True)

scaler = MinMaxScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

X = df[feature_columns].values[:-1]
y = df[feature_columns].values[1:]

split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

beta = np.random.randn(len(feature_columns) + 1, len(feature_columns))
learning_rate = 0.01
threshold = 1e-4
max_epochs = 1000

for epoch in tqdm(range(max_epochs), desc="Training Progress"):
    y_pred = np.dot(np.c_[np.ones(X_train.shape[0]), X_train], beta)
    error = y_pred - y_train
    mse = np.mean(error ** 2)

    gradient = (2 / len(X_train)) * np.dot(np.c_[np.ones(X_train.shape[0]), X_train].T, error)
    beta -= learning_rate * gradient

    y_test_pred = np.dot(np.c_[np.ones(X_test.shape[0]), X_test], beta)
    test_mse = np.mean((y_test_pred - y_test) ** 2)

    tqdm.write(f"Epoch {epoch + 1}: Train MSE={mse:.6f}, Test MSE={test_mse:.6f}")

    if mse < threshold:
        print("Stopping early as training error is below threshold.")
        break

print("Final Weights:", beta)



Dataset Columns: ['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max', 'BUDAPEST_cloud_cover', 'BUDAPEST_humidity', 'BUDAPEST_pressure', 'BUDAPEST_global_radiation', 'BUDAPEST_precipitation', 'BUDAPEST_sunshine', 'BUDAPEST_temp_mean', 'BUDAPEST_temp_max', 'DE_BILT_cloud_cover', 'DE_BILT_wind_speed', 'DE_BILT_wind_gust', 'DE_BILT_humidity', 'DE_BILT_pressure', 'DE_BILT_global_radiation', 'DE_BILT_precipitation', 'DE_BILT_sunshine', 'DE_BILT_temp_mean', 'DE_BILT_temp_min', 'DE_BILT_temp_max', 'DRESDEN_cloud_cover', 'DRESDEN_wind_speed', 'DRESDEN_wind_gust', 'DRESDEN_humidity', 'DRESDEN_global_radiation', 'DRESDEN_precipitation', 'DRESDEN_sunshine', 'DRESDEN_temp_mean', 'DRESDEN_temp_min', 'DRESDEN_temp_max', 'DUSSELDORF_cloud_cover', 'DUSSELDORF_wind_speed', 'DUSSELDORF_wind_gust', 'DUSSELDORF_humidity', 'DUSSELDORF_pressure', 'DUSSELDORF_global_r

Training Progress:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: Train MSE=49.882926, Test MSE=6.243806
Epoch 2: Train MSE=6.406363, Test MSE=4.909198


Training Progress:   1%|          | 6/1000 [00:00<00:35, 28.13it/s]

Epoch 3: Train MSE=5.064478, Test MSE=4.698189
Epoch 4: Train MSE=4.843059, Test MSE=4.532629
Epoch 5: Train MSE=4.665585, Test MSE=4.380999
Epoch 6: Train MSE=4.502582, Test MSE=4.241073


...

Epoch 988: Train MSE=0.539288, Test MSE=0.552780
Epoch 989: Train MSE=0.538944, Test MSE=0.552436
Epoch 990: Train MSE=0.538601, Test MSE=0.552091
Epoch 991: Train MSE=0.538257, Test MSE=0.551748


Training Progress:  99%|█████████▉| 994/1000 [00:53<00:00, 17.84it/s]

Epoch 992: Train MSE=0.537915, Test MSE=0.551405
Epoch 993: Train MSE=0.537573, Test MSE=0.551062
Epoch 994: Train MSE=0.537231, Test MSE=0.550720
Epoch 995: Train MSE=0.536890, Test MSE=0.550378


Training Progress: 100%|██████████| 1000/1000 [00:54<00:00, 18.50it/s]

Epoch 996: Train MSE=0.536549, Test MSE=0.550037
Epoch 997: Train MSE=0.536209, Test MSE=0.549696
Epoch 998: Train MSE=0.535869, Test MSE=0.549356
Epoch 999: Train MSE=0.535530, Test MSE=0.549016
Epoch 1000: Train MSE=0.535191, Test MSE=0.548677
Final Weights: [[ 1.13881582  0.30383753  0.82835183 ... -2.86406444 -0.32161383
   1.02501588]
 [-1.51646835  0.15135329 -0.39889649 ...  1.80728625 -0.12076481
   1.83917328]
 [-0.14811714  2.11100745 -1.02459078 ... -0.71920399 -0.15056644
   1.20302396]
 ...
 [ 0.57287024  0.33007624 -0.74918709 ...  0.61006595  1.01004855
  -1.89661203]
 [-0.23438432 -0.63474101 -0.52777305 ...  0.22380712 -0.40271538
  -1.17109881]
 [-1.63391897  0.95737078 -0.86049077 ... -0.31798714 -0.25871309
  -0.9389051 ]]


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

df = pd.read_csv("weather_prediction_dataset.csv")

print("Dataset Columns:", df.columns.tolist())

city = "BASEL"
feature_columns = [col for col in df.columns if col.startswith(f"{city}_")]

df = df[['DATE'] + feature_columns]

df['DATE'] = pd.to_datetime(df['DATE'])
df = df.sort_values(by='DATE')

df.fillna(df.mean(), inplace=True)

scaler = MinMaxScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

X = df[feature_columns].values[:-1]
y = df[feature_columns].values[1:]

split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1)
}

results = {}
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)

    results[name] = {
        "Train MSE": train_mse,
        "Test MSE": test_mse
    }
    print(f"{name}: Train MSE={train_mse:.6f}, Test MSE={test_mse:.6f}")

best_model = min(results, key=lambda x: results[x]["Test MSE"])
print("Best model:", best_model, "with Test MSE:", results[best_model]["Test MSE"])

Dataset Columns: ['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max', 'BUDAPEST_cloud_cover', 'BUDAPEST_humidity', 'BUDAPEST_pressure', 'BUDAPEST_global_radiation', 'BUDAPEST_precipitation', 'BUDAPEST_sunshine', 'BUDAPEST_temp_mean', 'BUDAPEST_temp_max', 'DE_BILT_cloud_cover', 'DE_BILT_wind_speed', 'DE_BILT_wind_gust', 'DE_BILT_humidity', 'DE_BILT_pressure', 'DE_BILT_global_radiation', 'DE_BILT_precipitation', 'DE_BILT_sunshine', 'DE_BILT_temp_mean', 'DE_BILT_temp_min', 'DE_BILT_temp_max', 'DRESDEN_cloud_cover', 'DRESDEN_wind_speed', 'DRESDEN_wind_gust', 'DRESDEN_humidity', 'DRESDEN_global_radiation', 'DRESDEN_precipitation', 'DRESDEN_sunshine', 'DRESDEN_temp_mean', 'DRESDEN_temp_min', 'DRESDEN_temp_max', 'DUSSELDORF_cloud_cover', 'DUSSELDORF_wind_speed', 'DUSSELDORF_wind_gust', 'DUSSELDORF_humidity', 'DUSSELDORF_pressure', 'DUSSELDORF_global_r

In [ ]:
df = pd.read_csv("weather_prediction_dataset.csv")

print("Dataset Columns:", df.columns.tolist())

cities = ["BASEL", "BUDAPEST", "DRESDEN"]

all_results = {}

for city in cities:
    print(f"\nProcessing city: {city}")

    feature_columns = [col for col in df.columns if col.startswith(f"{city}_")]

    if not feature_columns:
        print(f"Warning: No columns found for {city}. Skipping...")
        continue

    city_df = df[['DATE'] + feature_columns].copy()

    city_df['DATE'] = pd.to_datetime(city_df['DATE'])
    city_df = city_df.sort_values(by='DATE')

    city_df.fillna(city_df.mean(), inplace=True)

    scaler = MinMaxScaler()
    city_df[feature_columns] = scaler.fit_transform(city_df[feature_columns])

    X = city_df[feature_columns].values[:-1]
    y = city_df[feature_columns].values[1:]

    split_idx = int(0.8 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # Initialize models. if you put alpha's so low like 0.00001, the MSE's would be so close to eachother.
    models = {
        "Linear Regression": LinearRegression(),
        "Ridge Regression": Ridge(alpha=0.00001),
        "Lasso Regression": Lasso(alpha=0.00001)
    }

    results = {}
    for name, model in models.items():
        print(f"Training {name} for {city}...")
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        train_mse = mean_squared_error(y_train, y_train_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)

        results[name] = {
            "Train MSE": train_mse,
            "Test MSE": test_mse
        }
        print(f"{name} ({city}): Train MSE={train_mse:.6f}, Test MSE={test_mse:.6f}")

    all_results[city] = results

for city, results in all_results.items():
    best_model = min(results, key=lambda x: results[x]["Test MSE"])
    print(f"\nBest model for {city}: {best_model} with Test MSE: {results[best_model]['Test MSE']:.6f}")



Dataset Columns: ['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max', 'BUDAPEST_cloud_cover', 'BUDAPEST_humidity', 'BUDAPEST_pressure', 'BUDAPEST_global_radiation', 'BUDAPEST_precipitation', 'BUDAPEST_sunshine', 'BUDAPEST_temp_mean', 'BUDAPEST_temp_max', 'DE_BILT_cloud_cover', 'DE_BILT_wind_speed', 'DE_BILT_wind_gust', 'DE_BILT_humidity', 'DE_BILT_pressure', 'DE_BILT_global_radiation', 'DE_BILT_precipitation', 'DE_BILT_sunshine', 'DE_BILT_temp_mean', 'DE_BILT_temp_min', 'DE_BILT_temp_max', 'DRESDEN_cloud_cover', 'DRESDEN_wind_speed', 'DRESDEN_wind_gust', 'DRESDEN_humidity', 'DRESDEN_global_radiation', 'DRESDEN_precipitation', 'DRESDEN_sunshine', 'DRESDEN_temp_mean', 'DRESDEN_temp_min', 'DRESDEN_temp_max', 'DUSSELDORF_cloud_cover', 'DUSSELDORF_wind_speed', 'DUSSELDORF_wind_gust', 'DUSSELDORF_humidity', 'DUSSELDORF_pressure', 'DUSSELDORF_global_r

In [ ]:
# 3 other cities

df = pd.read_csv("weather_prediction_dataset.csv")

selected_cities = ["MAASTRICHT", "MUENCHEN", "TOURS"]
selected_features = ["pressure", "humidity", "precipitation"]
feature_columns = [f"{city}_{feature}" for city in selected_cities for feature in selected_features]

for col in feature_columns:
    if col not in df.columns:
        raise ValueError(f"Missing column: {col}")

df = df[['DATE'] + feature_columns].copy()

df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')
df = df.sort_values(by='DATE')

df.fillna(df.mean(), inplace=True)

scaler = MinMaxScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

city = "MUENCHEN"
city_features = [f"{city}_{feature}" for feature in selected_features]

X = df[city_features].values[:-1]
y = df[city_features].values[1:]

split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1),
    "Lasso Regression": Lasso(alpha=0.1)
}

results = {}
for name, model in models.items():
    print(f"Training {name} for {city}...")
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)

    results[name] = {
        "Train MSE": train_mse,
        "Test MSE": test_mse
    }
    print(f"{name}: Train MSE={train_mse:.6f}, Test MSE={test_mse:.6f}")

best_model = min(results, key=lambda x: results[x]["Test MSE"])
print(f"\nBest model for {city}: {best_model} with Test MSE: {results[best_model]['Test MSE']:.6f}")


Training Linear Regression for MUENCHEN...
Linear Regression: Train MSE=0.008241, Test MSE=0.007345
Training Ridge Regression for MUENCHEN...
Ridge Regression: Train MSE=0.008243, Test MSE=0.007352
Training Lasso Regression for MUENCHEN...
Lasso Regression: Train MSE=0.017098, Test MSE=0.015215

Best model for MUENCHEN: Linear Regression with Test MSE: 0.007345
